## Transfer Learning with MobileNet 

#### We will customise MobileNet, a lighweight, pretrained CNN for general image recognition, and use it for the most essential classification problem that ever existed - `dogs vs cats!`

---

#### Objectives:
* Test Mobilenet first
* Prepare some new dataset of images
* Create a shaved mobilenet, missing output layers
* Add our new layers 
* Freeze the training so it trains the new layers only
* wrap this into a new model class
* train and test our new model
* when we're happy, save the architecture (json) and the weights (h5)

---

### Imports

In [ ]:
import tensorflow.keras as keras
from keras import backend as K
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.optimizers import Adam
from keras.layers import Input, Dense, Activation
#from keras.utils.np_utils import to_categorical
from keras.applications.mobilenet import preprocess_input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2
import tqdm

In [ ]:
!python --version

In [ ]:
!pip freeze | grep opencv

#### Reset the session when you wish to reinitialise a different architecture

In [ ]:
K.clear_session()
keras.__version__

#### Instantiate Mobilenet

In [ ]:
mobile = MobileNet()

#### Prepare an image for MobileNet
* read into an array
* ensure the shape is correct
* for both steps, we can use a mixture of `opencv` and `numpy`

#### All our images need to have a shape of 224,224,3

In [ ]:
def process_for_mobilenet(file):
    # read the image into an tensor
    image = cv2.imread(file)
    # resize the tensor
    image = cv2.resize(image, dsize=(224,224),interpolation=cv2.INTER_CUBIC)
    # normalise the values of the tensor
    image = preprocess_input(image)
    return image

#### We're going to add a dimension so that mobilenet accepts it

In [ ]:
rabbit = process_for_mobilenet('/home/tommu/Downloads/rabbit.jpg')
plt.imshow(rabbit)

In [ ]:
print(rabbit.shape)
rabbit = np.expand_dims(rabbit,axis=0)
rabbit.shape

#### Test Mobilenet on the image to test it works

In [ ]:
rabbit.shape

In [ ]:
predictions = mobile.predict(rabbit)

In [ ]:
results = imagenet_utils.decode_predictions(predictions)
results

#### Now onto the real task, Transfer Learning

---

### Part 1: Chop and Customise

#### We chop when we import

In [ ]:
new_mobile = MobileNet(weights='imagenet',include_top=False, input_shape=(224,224,3))

#### And check the layers

In [ ]:
new_mobile.summary()

In [ ]:
len(new_mobile.layers)

#### Now add our own custom layers - the syntax here is different from the usual keras model creation
* First create your new_layer variable
* Then build a Keras `Model`, with inputs = the old mobilenet, and outputs = the new layers

In [ ]:
new_layers = new_mobile.output
new_layers = GlobalAveragePooling2D()(new_layers)
new_layers = Dense(1024, activation='relu')(new_layers)
new_layers = Dense(512, activation='relu')(new_layers)
new_layers = Dense(512, activation='relu')(new_layers)
new_layers = Dense(2, activation='softmax')(new_layers) # this is the only 'required' customisation - same no of neurons as classes

In [ ]:
new_mobile = Model(inputs=new_mobile.inputs, outputs=new_layers)

### 2: Freeze the layer weights that we don't want to train

In [ ]:
len(new_mobile.layers)

In [ ]:
for i, layer in enumerate(new_mobile.layers):
    if i < 87:
        layer.trainable = False
        
    else:
        layer.trainable = True

### 3: Prepare our images

#### Read in each image, and resize and preprocess it. We can also create our y data at the same time

In [ ]:
X = []
y = []
path = '/home/tommu/code/spiced/data/train_short/'
train_data = os.listdir(path)
train_data = train_data[:100]

for picture in tqdm.tqdm(train_data):
    image = process_for_mobilenet(path+picture)
    X.append(image)
    if 'cat' in picture:
        y.append(0)
    elif 'dog' in picture:
        y.append(1) 
X = np.array(X)
y = np.array(y)

#### The overall shape of X needs to be (no.of.images, 224,224,3), and y needs to be (no.of.images,no.of_classes)
* The shape of X should already be ok (but use `np.expand_dims` if you have one pic only)
* But we can reshape y using `keras.utils.np_utils.to_categorical`

In [ ]:
X.shape, y.shape

In [ ]:
y = to_categorical(y, num_classes=2)

In [ ]:
y[0]

#### Train-test split the data

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y)

### 4: Train and test!
#### Compile, fit and evaluate

In [ ]:
new_mobile.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
results = new_mobile.fit(Xtrain,ytrain, epochs=5, batch_size=128, validation_split=0.2)

In [ ]:
results = new_mobile.predict(Xtest)

In [1]:
from sklearn.metrics import accuracy_score
accuracy_score(results.round(), ytest)

NameError: name 'results' is not defined

---

#### Further work:
* Save the model (json) with its current weights (h5) to disk
* And load from disk

#### Save the model

In [ ]:
with open("model.json", "w") as json_file:
   json_file.write(new_mobile.to_json())
# serialize weights to HDF5
new_mobile.save_weights("model.h5")

#### Load the model

In [ ]:
from keras.models import model_from_json

# load json and create model
with open('model.json', 'r') as f:
   json = f.read()
model = model_from_json(json)

# load weights into new model
model.load_weights("model.h5")

In [ ]:
results = model.predict(Xtest)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(results.round(), ytest)